In [ ]:
pip install snowflake-connector-python pandas langchain_community langchain_text_splitters beautifulsoup4 pymilvus langchain_milvus langchain_huggingface huggingface_hub langchain_openai requests python-dotenv

In [1]:
# only execute once!!
#!cat 2022-IT-Root-CA.pem >> `python -m certifi`

In [100]:
import os
import mailbox
import re

import pandas as pd

def clean_addresses(addresses, lookupcsv):
    if addresses is None:
        return []
#   print(addresses)
    addresses = addresses.replace("\'", "")
    addressList = re.split('[,;]', addresses)
    cleanList = []
    for address in addressList:
        cleanAddress = clean_address(address, lookupcsv)
        cleanList.append(cleanAddress)
#       print(cleanAddress)
    return cleanList

def clean_address(address, lookupcsv):
#   print('Dirty:\t' + address)
    address = address.replace("<", "")
    address = address.replace(">", "")
    address = address.replace("\"", "")
    address = address.replace("\n", " ")
    address = address.replace("MAILER-DAEMON", "")
    address = address.lower().strip()

    with open(lookupcsv, 'rt') as lookupfile:
        lookupdata = lookupfile.readlines()
    for line in lookupdata:
        name = line.split(',')[0]
        if address == name:
            address = line.split(',')[-1].strip()

    email = None
    for word in address.split(' '):
        emailRegex = re.compile(
            "^[a-zA-Z0-9._%-]+@[a-zA-Z0-9._%-]+.[a-zA-Z]{2,6}$"
            )
        email = re.match(emailRegex, word)
        if email is not None:
            cleanEmail = email.group(0)
    if email is None:
        if address.split(' ')[-1].find('@') > -1:
            cleanEmail = address.split(' ')[-1].strip()
        elif address.split(' ')[-1].find('?') > -1:
            cleanEmail = 'n/a'
        else:
            cleanEmail = address
              
#    print('Clean:\t' + cleanEmail)
    return cleanEmail

def get_body(message):
    try:
        sm = str(message)
        body_start = sm.find('iamunique', sm.find('iamunique')+1)
        body_start = sm.find('Content-Transfer-Encoding', body_start+1)
        body_start = sm.find('\n', body_start+1)+1
    
        body_end = sm.find('From: ', body_start + 1)
        if body_end == -1:
            body_end = sm.find('iamunique', body_start + 1)
            body_end = sm.find('\n', body_end - 25)
        body = sm[body_start:body_end]

        body = body.replace("=20\n", "")
        body = body.replace("=FC", "ü")
        body = body.replace("=F6", "ö")
        body = body.replace("=84", "\"")
        body = body.replace("=94", "\"")
        body = body.replace("=96", "-")
        body = body.replace("=92", "\'")
        body = body.replace("=93", "\"")
        body = body.replace("=E4", "ä")
        body = body.replace("=DF", "ss")
        body = body.replace("=", "")
        body = body.replace("\"", "")
        body = body.replace("\'", "")
    except:
        body = "N/A"
    #print("---<<----")
    #print(body)
    #print()
    
    return body

def write_table(mboxfile, mailTable, pathToCleanup):
    for message in mailbox.mbox(mboxfile):
        cleanFrom = clean_address(message['From'], pathToCleanup)
        cleanTo = clean_addresses(message['To'], pathToCleanup)
        cleanCc = clean_addresses(message['Cc'], pathToCleanup)
        mailTable.append([
            cleanFrom,
            cleanTo,
            cleanCc,
            message['Date'],
            message['Subject'],
            get_body(message)
            ])
   
pathToEmails  = './'
pathToCleanup = './cleanup.csv'
mboxfiles = [os.path.join(dirpath, f)
	     for dirpath, dirnames, files in os.walk(pathToEmails)
	     for f in files if f.endswith('mbox')]
mailTable = []
#print(mboxfiles)

for mboxfile in mboxfiles:
#   print(mboxfile)
    write_table(mboxfile, mailTable, pathToCleanup)

m = pd.DataFrame(mailTable)
m.columns = ['From', 'To', 'Cc', 'Date', 'Subject', 'Body']


In [101]:
display(m)

,From,To,Cc,Date,Subject,Body
0,aleksandr.milovidov@raiffeisen.ru.invalid,[users@activemq.apache.org],[],"Tue, 1 Oct 2024 08:04:05 +0000",Messages are stuck in the paging,MIME-Version: 1.0\nX-KSMG-AntiPhishing: not sc...
1,jason.jackson@itechag.com.invalid,[users@activemq.apache.org],[],"Wed, 2 Oct 2024 12:54:02 +0000",Re: ActiveMQ LDAP Query objectClass Issue,\nVGhhbmsgeW91IGZvciB0aGUgaW5mb3JtYXRpb24uDQoN...
2,jb@nanthrax.net,"[dev@activemq.apache.org, users@activemq.apach...",[],"Wed, 2 Oct 2024 17:47:48 +0200",[ANN] Apache ActiveMQ 5.18.6 has been released!,X-GND-Sasl: jb@nanthrax.net\n\nThe ActiveMQ te...
3,christopher.edwards@hibu.com.invalid,[users@activemq.apache.org],[],"Wed, 2 Oct 2024 22:49:03 +0000",Problem with pathSpec configuration in jetty.xml,"\nHi,\n\nI recently started using activeMQ 6.1..."
4,baigildin97@gmail.com,[users@activemq.apache.org],[],"Thu, 3 Oct 2024 10:16:44 +0300",[Artemis] Passing message userID into STOMP,X-Original-To: archive-asf-public@cust-asf.pon...
...,...,...,...,...,...,...
1068,vilius.sumskas@rivile.lt.invalid,[users@activemq.apache.org],[],"Thu, 17 Apr 2025 11:55:32 +0000",RE: IP address whitelisting for Artemis users,MIME-Version: 1.0\nX-OriginatorOrg: rivile.lt\...
1069,bruscinodf@gmail.com,[users@activemq.apache.org],[],"Thu, 17 Apr 2025 15:24:22 +0200",Re: IP address whitelisting for Artemis users,"\nYes, you can use a connection-router to allo..."
1070,bruscinodf@gmail.com,[users@activemq.apache.org],[],"Thu, 17 Apr 2025 15:25:25 +0200",Re: IP address whitelisting for Artemis users,"\nIn my previous example there is an error, I ..."
1071,milovidoff@gmail.com,[users@activemq.apache.org],[],"Thu, 17 Apr 2025 16:41:03 +0300",Re: limiting queue creation in JMS durable sub...,"\nThe producer is not aware of consumers, so w..."


In [4]:
from dotenv import load_dotenv
from langchain_core.documents import Document

In [112]:
from uuid import uuid4
documents = [
        Document(page_content="Sender: " + From +"\n" + "Date: " + Date + "\n" + "Subject: " + Subject+ "\n\n" + Body, metadata={"Sender": From, "Date": Date, "Subject": Subject}, id=uuid4()) 
        for Body, From, To, Date, Subject in zip(m['Body'], m['From'], m['To'], m['Date'], m['Subject'])
    ]
print(len(documents))

1073


In [70]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_milvus import Milvus
from dotenv import load_dotenv
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
# Initialize chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

chunked_docs = text_splitter.split_documents(documents)

In [115]:
# Initialize HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

# Store in Milvus
vectorstore = Milvus.from_documents(  
    documents=documents,
    embedding=embeddings,
    connection_args={"uri": "./milvus_demo.db"},  
    drop_old=False, 
    index_params={"index_type": "FLAT", "metric_type": "L2"},
)

print("Successfully stored embeddings in Milvus!")

Successfully stored embeddings in Milvus!


In [72]:
import getpass
access_token = getpass.getpass("Enter your MODELS.CORP API KEY for granite-3.1-8b-instruct")
model_api_url = "https://granite-3-1-8b-instruct--apicast-production.apps.int.stc.ai.prod.us-east-1.aws.paas.redhat.com/v1"
model = "/data/granite-3.1-8b-instruct"

Enter your MODELS.CORP API KEY for granite-3.1-8b-instruct ········


In [116]:
llm = ChatOpenAI(model=model, api_key=access_token, base_url=model_api_url, temperature=0.1)

# Define the prompt template
PROMPT_TEMPLATE = """
Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a complete answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

# Create the prompt template
prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create the chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [138]:
query = "Summarize what was discussed on the topic 'Additional Info on certificate based authentication errors'. Who is involved in this discussion?"

res = rag_chain.invoke(query)
print("--------------------------\n")
print("Question : ",query)
print("\n--------------------------\n")
print("Response : ",res)
print("\n--------------------------")

--------------------------

Question :  Summarize what was discussed on the topic 'Additional Info on certificate based authentication errors'. Who is involved in this discussion?

--------------------------

Response :  The discussion revolves around certificate-based authentication errors in the context of Apache ActiveMQ Artemis. The participants in this discussion are Justin, Shiv Kumar Dixit, and Domenico.

Justin initially inquires whether Shiv wants to extract certificate data on the client or the broker. He points out that the broker already has code to get certificate data, but the SSL handshake fails, preventing the broker from inspecting the certificate data on the connection.

Shiv responds by sharing a StackOverflow link that suggests extracting SSL information from the ChannelHandlerContext object in Netty. He provides a code snippet demonstrating how to access the peer certificate chain.

Domenico then enters the conversation, advising Shiv to add 'netty-tcnative' to the